In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
import torch
from botorch.models import SingleTaskGP, ModelListGP, fully_bayesian
from botorch.models.gp_regression_mixed import MixedSingleTaskGP
from botorch.fit import fit_gpytorch_mll, fit_fully_bayesian_model_nuts
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition.monte_carlo import qExpectedImprovement, qUpperConfidenceBound
from botorch.acquisition.analytic import UpperConfidenceBound, ProbabilityOfImprovement, ExpectedImprovement
from botorch.optim import optimize_acqf, optimize_acqf_mixed
from botorch.cross_validation import gen_loo_cv_folds
import math
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, Matern, DotProduct, RationalQuadratic, WhiteKernel
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [2]:
train_X = torch.cat([torch.rand(20, 2), torch.randint(3, (20, 1))], dim=-1)
train_X

tensor([[0.6893, 0.6896, 2.0000],
        [0.1220, 0.8342, 2.0000],
        [0.3268, 0.4434, 1.0000],
        [0.6981, 0.8052, 0.0000],
        [0.5954, 0.7287, 2.0000],
        [0.3218, 0.8583, 1.0000],
        [0.5602, 0.9682, 2.0000],
        [0.4828, 0.8117, 2.0000],
        [0.9875, 0.9875, 2.0000],
        [0.6173, 0.7877, 1.0000],
        [0.1799, 0.1028, 1.0000],
        [0.0885, 0.2167, 1.0000],
        [0.4714, 0.0359, 2.0000],
        [0.3904, 0.1892, 1.0000],
        [0.6177, 0.5971, 1.0000],
        [0.8983, 0.4819, 0.0000],
        [0.1785, 0.8136, 2.0000],
        [0.4199, 0.5085, 0.0000],
        [0.1043, 0.5190, 1.0000],
        [0.5869, 0.8326, 2.0000]])

In [3]:
train_Y = (
        torch.sin(train_X[..., :-1]).sum(dim=1, keepdim=True)
        + train_X[..., -1:]
    )
train_Y

tensor([[3.2722],
        [2.8624],
        [1.7500],
        [1.3637],
        [3.2267],
        [2.0730],
        [3.3552],
        [3.1897],
        [3.6693],
        [2.2875],
        [1.2815],
        [1.3034],
        [2.4900],
        [1.5686],
        [2.1414],
        [1.2458],
        [2.9044],
        [0.8945],
        [1.6001],
        [3.2935]])

In [4]:
model = MixedSingleTaskGP(train_X, train_Y, cat_dims=[-1])

/Users/yimengjin/Desktop/Thesis/Thesis_Yimeng/venv/lib/python3.9/site-packages/botorch/models/gpytorch.py:113: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(


In [5]:
EI = qExpectedImprovement(
    model = model,
    best_f = max(train_Y),
)

In [11]:
X_candidates,_ = optimize_acqf_mixed(
    acq_function = EI,
    bounds = torch.tensor([[0., 0., 0.,],[1., 1., 1.]]),
    q = 20, # Number of suggested candidates, 20 candidates took around 40 mins
    num_restarts = 200,
    raw_samples = 512,
    # constraints = [(torch.tensor([1, 2, 3, 4]), torch.tensor([1.]*4), 1.0)],
    fixed_features_list = [{2:1},{2:0},{2:3}]
)
X_candidates

/Users/yimengjin/Desktop/Thesis/Thesis_Yimeng/venv/lib/python3.9/site-packages/botorch/optim/optimize.py:306: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2.')]
Trying again with a new set of initial conditions.
  warnings.warn(first_warn_msg, RuntimeWarning)
/Users/yimengjin/Desktop/Thesis/Thesis_Yimeng/venv/lib/python3.9/site-packages/botorch/optim/optimize.py:306: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2.')]
Trying again with a new set of initial conditions.
  warnings.warn(first_warn_msg, RuntimeWarning)
/Users/yimengjin/Desktop/Thesis/Thesis_Yimeng/venv/lib/python3.9/site-packages/botorch/optim/optimize.py:306: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[Optimi

tensor([[0.9835, 1.0000, 3.0000],
        [0.5328, 1.0000, 3.0000],
        [1.0000, 0.7049, 3.0000],
        [1.0000, 0.1512, 3.0000],
        [1.0000, 0.4569, 3.0000],
        [0.7300, 0.4558, 3.0000],
        [0.9982, 0.7034, 3.0000],
        [0.0081, 1.0000, 3.0000],
        [1.0000, 1.0000, 1.0000],
        [0.9922, 0.2922, 3.0000],
        [0.7786, 0.9393, 3.0000],
        [0.9862, 0.0000, 3.0000],
        [0.3912, 1.0000, 3.0000],
        [0.8414, 0.6771, 3.0000],
        [0.9377, 0.9423, 3.0000],
        [0.0000, 0.0000, 3.0000],
        [0.9410, 0.9930, 3.0000],
        [0.5306, 0.9996, 3.0000],
        [0.1901, 0.7261, 3.0000],
        [0.6686, 0.8874, 3.0000]])